# Load Dependencies

In [ ]:
import pandas as pd
import numpy as np
import math
import joblib
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from torch.utils.tensorboard import SummaryWriter

# Mount Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Set device as GPU if available

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Project 1 (4-Neighbor Agent)


##Model Training Function

In [ ]:
def train_model(data_loader, model, loss_fn, optimizer, epochs):
    train_dataset_size = len(data_loader.dataset)
    num_batches = len(data_loader)
    model.train()
    for epoch in range(epochs):
        train_loss, correct = 0, 0
        print(f'Start of epoch [{epoch+1}/{epochs}]')
        for batch_number, data in enumerate(data_loader):
            X, y = data
            # Compute prediction and loss
            probabilities = model(X)
            loss = loss_fn(probabilities, y)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Compute and display training loss and number of correct predictions at each epoch
            prediction = torch.argmax(probabilities, axis=1)
            train_loss += loss.item() 
            correct += (prediction == y).sum().item()

            if batch_number % 100 == 0:
                print(f'Batch [{batch_number+1}/{num_batches}] --> Accuracy: {(correct/(64*(batch_number+1))):.4f}')

            # For every 3 batches of data trained on write data for tensorboard
            if batch_number % 3 == 0:
                train_writer.add_scalar('loss', (train_loss/(64*(batch_number+1))), (epoch * train_dataset_size) + (64*(batch_number + 1)))
                train_writer.add_scalar('accuracy', (correct/(64*(batch_number+1))), (epoch * train_dataset_size) + (64*(batch_number + 1)))
          
        print(f'End of epoch [{epoch+1}/{epochs}], Loss: {train_loss/train_dataset_size:.4f}, Accuracy: {(correct/train_dataset_size):.4f}')
    print("Done Training!")

##Model Testing Function

In [ ]:
def test_model(data_loader, model, loss_fn):
    test_dataset_size = len(test_loader.dataset)
    num_batches = len(test_loader)
    with torch.no_grad():
        test_loss, correct = 0, 0
        for batch_number, data in enumerate(test_loader):
            X, y = data
            # Compute prediction and loss
            probabilities = model(X)
            loss = loss_fn(probabilities, y)

            # Compute and display testing loss and number of correct predictions 
            prediction = torch.argmax(probabilities, axis=1)
            test_loss += loss.item() 
            correct += (prediction == y).sum().item()

            if batch_number % 3 == 0:
                print(f'Batch [{batch_number+1}/{num_batches}] --> Accuracy: {(correct/(64*(batch_number+1))):.4f}')
                test_writer.add_scalar('accuracy', (correct/(64*(batch_number+1))), (64*(batch_number + 1)))
          
    print(f'Final Test Data Loss: {test_loss/test_dataset_size:.4f}, Accuracy: {(correct/test_dataset_size):.4f}')
    print("Done!")

##Load Train/Test Datasets and Convert to Tensors

In [ ]:
class GridWorldDataset(Dataset):

    def __init__(self, file, transform=torch.tensor):
        data = joblib.load(file)
        df = pd.DataFrame(data, columns=['marked_agent_grid', 'move'])
        self.gridview_x = df['marked_agent_grid']
        self.move_y = df['move']
        self.transform = transform

    def __len__(self):
        return len(self.move_y)

    def __getitem__(self, index):
        x, y = self.gridview_x.iloc[index], self.move_y.iloc[index]
        x, y = self.transform(x), self.transform(y)
        return x,y

In [ ]:
#Train data loader
train_data_path = '/content/drive/MyDrive/train_data'
train_data = GridWorldDataset(file=train_data_path, transform=torch.tensor)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

#Test data loader
test_data_path = '/content/drive/MyDrive/test_data'
test_data = GridWorldDataset(file=test_data_path, transform=torch.tensor)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True)

##Simple Neural Net

### Simple Neural Net Model Architecture

In [ ]:
# Neural net architecture
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.flatten = nn.Flatten()
        self.l1 = nn.Linear(64, 32) 
        self.l2 = nn.Linear(32, 16) 
        self.l3 = nn.Linear(16, 16) 
        self.l4 = nn.Linear(16, 4)  
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()

    def forward(self, x):
        x = self.flatten(x.float())
        x = self.l1(x)
        x = self.relu(x)
        x = self.l2(x)
        x = self.relu(x)
        x = self.l3(x)
        x = self.relu(x)
        x = self.l4(x)
        logits = self.relu(x)
        probabilities = self.softmax(logits)
        return probabilities

# Model of NN
model = NeuralNet().to(device)

###Training Simple Neural Net Model

In [ ]:
train_writer = SummaryWriter('runs/proj1/nn/train')
train_model(train_loader, model, loss_fn = nn.CrossEntropyLoss(), optimizer = torch.optim.Adam(model.parameters(), lr=0.0001), epochs = 100)

### Save Model

In [ ]:
PATH='model/proj1/nn.pt'
torch.save(model.state_dict(), PATH)

### Load Model

In [ ]:
model = NeuralNet()
PATH='model/proj1/nn.pt' 
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

###Testing Simple Neural Net Model

In [ ]:
test_writer = SummaryWriter('runs/proj1/nn/test')
test_model(test_loader, model, loss_fn = nn.CrossEntropyLoss())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Batch [1/316] --> Accuracy: 0.9219
Batch [4/316] --> Accuracy: 0.9336
Batch [7/316] --> Accuracy: 0.9129
Batch [10/316] --> Accuracy: 0.9156
Batch [13/316] --> Accuracy: 0.9171
Batch [16/316] --> Accuracy: 0.9111
Batch [19/316] --> Accuracy: 0.9104
Batch [22/316] --> Accuracy: 0.9112
Batch [25/316] --> Accuracy: 0.9137
Batch [28/316] --> Accuracy: 0.9096
Batch [31/316] --> Accuracy: 0.9093
Batch [34/316] --> Accuracy: 0.9104
Batch [37/316] --> Accuracy: 0.9092
Batch [40/316] --> Accuracy: 0.9109
Batch [43/316] --> Accuracy: 0.9117
Batch [46/316] --> Accuracy: 0.9096
Batch [49/316] --> Accuracy: 0.9107
Batch [52/316] --> Accuracy: 0.9105
Batch [55/316] --> Accuracy: 0.9105
Batch [58/316] --> Accuracy: 0.9095
Batch [61/316] --> Accuracy: 0.9096
Batch [64/316] --> Accuracy: 0.9092
Batch [67/316] --> Accuracy: 0.9074
Batch [70/316] --> Accuracy: 0.9071
Batch [73/316] --> Accuracy: 0.9077
Batch [76/316] --> Accuracy: 0.9083
Batch [79/316] --> Accuracy: 0.9086
Batch [82/316] --> Accuracy: 0.

###Tensorboard data analysis

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

##Convulational Neural Net

### Convulational Neural Net Model Architecture

In [ ]:
# Convulational Neural net architecture
class ConvulationalNeuralNet(nn.Module):
    def __init__(self):
        super(ConvulationalNeuralNet, self).__init__()
        self.flatten = nn.Flatten()
        self.l1 = nn.Conv2d(1, 32, kernel_size=2, stride=1, padding=0) 
        self.l2 = nn.Conv2d(32, 64, kernel_size=2, stride=1, padding=0) 
        self.l3 = nn.Conv2d(64, 64, kernel_size=2, stride=1, padding=0)
        self.l4 = nn.Conv2d(64, 32, kernel_size=2, stride=1, padding=0) 
        self.l5 = nn.Conv2d(32, 16, kernel_size=2, stride=1, padding=0) 
        self.l6 = nn.Linear(144, 4) 
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()

    def forward(self, x):
        x = torch.unsqueeze(x.float(), 1)
        x = self.l1(x)
        x = self.relu(x)
        x = self.l2(x)
        x = self.relu(x)
        x = self.l3(x)
        x = self.relu(x)
        x = self.l4(x)
        x = self.relu(x)
        x = self.l5(x)
        x = self.relu(x)
        x = self.flatten(x)
        x = self.l6(x)
        logits = self.relu(x)
        probabilities = self.softmax(logits)
        return probabilities

# Model of CNN
cnn_model = ConvulationalNeuralNet().to(device)

###Training Convulational Neural Net Model

In [ ]:
train_writer = SummaryWriter('runs/proj1/cnn/train')
train_model(train_loader, cnn_model, loss_fn = nn.CrossEntropyLoss(), optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.001), epochs = 50)
train_model(train_loader, cnn_model, loss_fn = nn.CrossEntropyLoss(), optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.0001), epochs = 50)

### Save Model

In [ ]:
PATH='model/proj1/cnn.pt'
torch.save(cnn_model.state_dict(), PATH)

### Load Model

In [ ]:
cnn_model = ConvulationalNeuralNet()
PATH='model/proj1/cnn.pt' 
cnn_model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

###Testing Convulational Neural Net Model

In [ ]:
test_writer = SummaryWriter('runs/proj1/cnn/test')
test_model(test_loader, cnn_model, loss_fn = nn.CrossEntropyLoss())

###Tensorboard data analysis

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

## 4-Neighbor Agent Analysis


### Utils

In [ ]:
from collections import defaultdict
import numpy as np
import heapq as hq
import copy
import random
import math
import csv
import joblib
import time

class Cell:
    def __init__(self, x, y, state=0, fval=math.inf, gval=math.inf, hval=0, parent=(-10, -10)):
        self.x = x
        self.y = y
        self.hval = hval
        self.fval = fval
        self.gval = gval
        self.parent = parent
        self.state = state

def generate_gridworld(dim, probability=0.0):
    '''
    Generate the gridworld of size [dim, dim]
    0 is unblocked 1 is blocked
    Each cell is blocked with probability p and unblocked with probability 1-p
    Takes the dimension (n) and value of prbability p as input 
    and returns a nxn gridworld
    '''
    # random.seed(100) #for reproducible results
    gridworld =  np.random.choice([1, 0], size=(dim,dim), p=[probability, 1-probability]) 
    gridworld[0][0]=0
    gridworld[dim-1][dim-1]=0
    return gridworld

### Original AI Agent Implementation

In [ ]:
class PathFinder:
    def __init__(self, gridworld, agent_grid_view, dim, start, goal):
        """
        Builder function
        Takes Paramets: gridworld, agent_grid_view, belief_state, dim, start, target, agent
        """
        self.gridworld = gridworld
        self.dim = dim
        self.agent_grid_view = agent_grid_view
        self.start = start
        self.goal = goal  
        self.cells = {}  # Initialize Cells Hash
        self.children_hash = {}  # Initiliaze Children Hash
        self.blocks_encountered = 0  # keep track of blocks encountered
        self.replans_count = 0  # Counter to keep track of no of replans
        self.cells_processed=0
        self.examination_cost = 0  # No of nodes actually examined
        # create cell objects
        for i in range(0, dim):
            for j in range(0, dim):
                self.cells[(i, j)] = Cell(i, j)

    def heuristic(self, x1, y1):
        """
        Function used to calculate Manhattan Distance. 
        Takes row,col as input and returns the Manhattan Distance
        """
        x2, y2 = self.goal
        return abs(x1-x2) + abs(y1-y2)

    def get_children(self, cell_x, cell_y):
        """
        function used to get the next valid reachable cells from given cell
        Takes row,col as input and returns the next possible children of given cell
        """
        if self.children_hash.get((cell_x, cell_y)) != None:
            return self.children_hash.get((cell_x, cell_y))
        action_offset = [
            (1, 0),  # Right
            (0, -1),  # Down
            (-1, 0),  # Left
            (0, 1),  # Up
        ]
        children = []
        for n in action_offset:
            x = cell_x + n[0]
            y = cell_y + n[1]
            if 0 <= x < self.dim and 0 <= y < self.dim:
                children.append((x, y))
        self.children_hash[(cell_x, cell_y)] = children
        return children

    def astar(self, start_cell, goal_cell):
        """
        funciton to implement A* algorithm and caluclate path.
        Input start node and given goal node, returns path from start node to goal node and a boolean solvable to indicate if goal is reachable from the start
        """
        solvable = True
        fringe = []
        path = []
        goal = goal_cell
        start_cell = self.cells.get(start_cell)
        start_cell.fval = self.heuristic(start_cell.x, start_cell.y)
        start_cell.gval = 0
        start_cell.parent = None
        hq.heappush(fringe, (start_cell.fval, (start_cell.x, start_cell.y)))
        visited_list = {}
        self.cells_processed =0
        while len(fringe):
            current_cell = hq.heappop(fringe)
            self.cells_processed+=1
            current_cell = current_cell[1]
            if current_cell == goal:
                path = []
                current_cell = self.cells.get(
                    (current_cell[0], current_cell[1]))
                while current_cell != None:
                    path.append((current_cell.x, current_cell.y))
                    current_cell = self.cells.get(
                        (current_cell.x, current_cell.y)).parent
                return path[::-1], True
            elif visited_list.get(current_cell) == None:
                # add cell to closed list since we are exploring
                visited_list[current_cell] = current_cell
                children = self.children_hash.get(
                    (current_cell[0], current_cell[1]), self.get_children(current_cell[0], current_cell[1]))
                # iterate through them and add them to fringe if they are not in closed set
                parent = self.cells.get((current_cell[0], current_cell[1]))
                for child_x, child_y in children:
                    if self.agent_grid_view[child_x][child_y] == 1 or visited_list.get((child_x, child_y)) != None:
                        continue
                    else:
                        child = self.cells.get((child_x, child_y))
                        child.parent = parent
                        child.gval = self.cells.get(current_cell).gval+1
                        child.hval = self.heuristic(child_x, child_y)
                        child.fval = child.gval+child.hval
                        # add to fringe lowest one of multiple entries comes out
                        hq.heappush(fringe, (child.fval, (child_x, child_y)))
        solvable = False
        return path, solvable

    def replan_path(self, cell):
        """
        Function used to replan path, when encountered a block on traversing the path given by A*
        """
        self.replans_count += 1  # counter to count number of restarts
        self.blocks_encountered+=1
        restart_cell = self.cells.get(cell)
        restart_cell.parent = None
        restart_cell.gval = 0
        restart_cell.hval = self.heuristic(restart_cell.x, restart_cell.y)
        restart_cell.fval = restart_cell.hval
        return self.astar((restart_cell.x, restart_cell.y), self.goal)   

    def execute_agent(self, start):
        """
        Function that executes agent 10
        Takes input initial random start state
        Return the final path and cost metrics
        """
        # Initializing all values
        # print("Executing Agent", self.agent)
        start_cell = start
        self.cells.get(start_cell).parent = None
        goal_found = False
        complete_path = []
        # Find path from random start state to random goal state
        path, solvable = self.astar(start_cell, self.goal)
        time_step =0
        while solvable and goal_found !=True:
            for i,cell in enumerate(path):
                time_step+=1
                # sense knowledge 
                neighbors = self.get_children(cell[0],cell[1])
                for neighbor in neighbors:
                   x,y = neighbor
                   if self.gridworld[x][y] == 1:
                        self.agent_grid_view[x][y] = 1
                 # check for goal state
                if cell[0] == self.goal[0]  and cell[1] == self.goal[1]:
                    complete_path.append(cell)
                    goal_found = True
                    break
                else:
                    next_cell =  path[i+1] if i+1<len(path) else None
                    if next_cell!=None and  self.gridworld[next_cell[0]][next_cell[1]] == 1:
                            # revert to parent and replan
                            self.agent_grid_view[next_cell[0]][next_cell[1]] = 1
                            path,solvable = self.replan_path(cell)
                            break
                    else:
                        complete_path.append(cell)       
        if goal_found:
            # print('Hidden goal found')
            return 1,len(complete_path),complete_path, self.replans_count, self.cells_processed
        else:
            print('target not reachable')
            return 0,0,[],self.replans_count, self.cells_processed


### ML Agent Implementation

#### Inference function for ML agent

In [ ]:
def inference(grid, model):
  grid = torch.tensor(grid)
  grid = grid.float()
  grid = grid.unsqueeze(0)
  #grid = nn.Flatten()(grid)
  with torch.no_grad():
    model.eval()       
    probabilities = model(grid)
    probabilities = probabilities[0].to(device).detach().numpy()
  return probabilities

#### ML agent class


In [ ]:
class Mlfinder:
    def __init__(self, gridworld, agent_grid_view, dim, start, goal, ml_model):
        """
        Builder function
        Takes Paramets: gridworld, agent_grid_view, belief_state, dim, start, target, agent
        """
        self.model = ml_model
        self.gridworld = gridworld
        self.dim = dim
        self.agent_grid_view = agent_grid_view
        self.start = start
        self.goal = goal
        self.cells = {}  # Initialize Cells Hash
        self.children_hash = {}  # Initiliaze Children Hash
        self.blocks_encountered = 0  # keep track of blocks encountered
        self.replans_count = 0  # Counter to keep track of no of replans
        self.cells_processed=0
        self.examination_cost = 0  # No of nodes actually examined
        # create cell objects
        for i in range(0, dim):
            for j in range(0, dim):
                self.cells[(i, j)] = Cell(i, j)
   
    def get_children(self, cell_x, cell_y):
        """
        function used to get the next valid reachable cells from given cell
        Takes row,col as input and returns the next possible children of given cell
        """
        if self.children_hash.get((cell_x, cell_y)) != None:
            return self.children_hash.get((cell_x, cell_y))
        action_offset = [
            (1, 0),  # Right
            (0, -1),  # Down
            (-1, 0),  # Left
            (0, 1),  # Up
        ]
        children = []
        for n in action_offset:
            x = cell_x + n[0]
            y = cell_y + n[1]
            if 0 <= x < self.dim and 0 <= y < self.dim:
                children.append((x, y))
        self.children_hash[(cell_x, cell_y)] = children
        return children            

    def get_cell(self, cell_x, cell_y,dir):
        action_offset = {
           0: (1, 0),  # Right
            1:(0, -1),  # Down
            2:(-1, 0),  # Left
            3: (0, 1),  # Up
        }
        n = action_offset.get(dir)
        x = cell_x + n[0]
        y = cell_y + n[1]
        if 0 <= x < self.dim and 0 <= y < self.dim:
          if(self.gridworld[x][y]==1):
            return None
          return (x,y)
        else:
          return None  
   
    def slice_grid(self,dim,cell,agent_grid_view):
            agent_grid_view[cell[0]][cell[1]]=5
            padded_data = np.full((dim+10,dim+10),-1)
            for i in range(dim):
                 for j in range(dim):
                    padded_data[i+4][j+4] = agent_grid_view[i][j]
            x,y = cell
            slice = padded_data[x-3+4:x+5+4,y-3+4:y+5+4]
            if(slice.shape[0]<8 or slice.shape[1]<8):
                print('ERRRORRRR',slice.shape)
            return slice     
    
    def get_next_cell(self,current_cell):
      copy_agent_view = copy.deepcopy(self.agent_grid_view)
      local_grid = self.slice_grid(self.dim,current_cell,copy_agent_view)
      directions = inference(local_grid, self.model)
      return directions
    
    def contious_cells(self,direction,cell):
          cell_x,cell_y =cell
          action_offset = {
            0: (1, 0),  # Right
            1: (0, -1),  # Down
            2: (-1, 0),  # Left
            3: (0, 1),  # Up
                           }
          n = action_offset.get(direction)
          prediction = []
          for i in range(1):
               x = cell_x + n[0]*i
               y = cell_y + n[1]*i
               if 0 <= x < self.dim and 0 <= y < self.dim:
                    prediction.append((x,y))
          return prediction 

    def loop_cell(self,current_cell):
      state = True
      directions = self.get_next_cell(current_cell)
      cell = None
      no_direction = 0
      while no_direction < 4 and cell == None:
            no_direction += 1
            direction = np.argmax(directions)
            directions[direction] = -1
            cell = self.get_cell(current_cell[0],current_cell[1],direction)
      return self.contious_cells(direction,cell)if cell!=None else []    

    def execute_agent(self, start):
        """
        Function that executes agent 10
        Takes input initial random start state
        Return the final path and cost metrics
        """
        # Initializing all values
        # print("Executing Agent", self.agent)
        start_cell = start
        self.cells.get(start_cell).parent = None
        goal_found = False
        complete_path = [start]
        # Find path from random start state to random goal state 
        path = self.loop_cell(start)

        time_step =0
        while  goal_found !=True and len(path)>0 and time_step<500:
            for i,cell in enumerate(path):
                time_step+=1

                if self.gridworld[cell[0]][cell[1]]==0:
                  # sense knowledge 
                   neighbors = self.get_children(cell[0],cell[1])
                   for neighbor in neighbors:
                     x,y = neighbor
                     if self.gridworld[x][y] == 1:
                          self.agent_grid_view[x][y] = 1
                 # check for goal state
                if cell[0] == self.goal[0]  and cell[1] == self.goal[1]:
                    complete_path.append(cell)
                    goal_found = True
                    break
                else:
                    if self.gridworld[cell[0]][cell[1]] == 1:
                            # revert to parent and replan
                            self.agent_grid_view[cell[0]][cell[1]] = 1
                            prev = complete_path.pop()
                            path= self.loop_cell(prev)
                            break
                    else:
                        complete_path.append(cell)
                        path=self.loop_cell(cell) 
                        
        if goal_found:
            print('goal found by ML')
            return 1,len(complete_path),complete_path, self.replans_count, self.cells_processed
        else:
            print('target not reachable')
            return 0,0,[],self.replans_count, self.cells_processed

### Execute both agents

In [300]:
from time import process_time

def test_ml_agent(number_of_mazes):
  dim = 50
  start =(0,0)
  goal =(dim-1,dim-1)

  '''[[agent_solved, agent_unsolved],
    [ml_agent_solved, ml_agent_unsolved]]'''
  confusion_matrix = np.array([[0, 0], 
                        [0, 0]])
  total_path_length = 0
  total_run_time = 0
  total_ml_path_length = 0
  total_ml_run_time = 0

  solved_agent =0
  solved_ml_agent =0
  for i in range(number_of_mazes):
    print(i,'sample------------------')
    gridworld= generate_gridworld(dim, 0.3)

    # start = process_time()
    goal_found_agent,path_length,_, _, _ = PathFinder(gridworld, np.zeros((dim, dim)), dim, start, goal).execute_agent(start)

    solved_agent+=goal_found_agent 
    # runtime = process_time() - start

    ml_model = ConvulationalNeuralNet()
    PATH='model/proj1/cnn.pt' 
    ml_model.load_state_dict(torch.load(PATH))
    # start_ml = process_time()
    goal_found_ml, ml_path_length,_, _, _ = Mlfinder(gridworld, np.zeros((dim, dim)), dim, start, goal, ml_model).execute_agent(start)
    # ml_runtime = process_time() - start_ml
    solved_ml_agent+=goal_found_ml
    
    total_path_length += path_length
    total_ml_path_length += ml_path_length
    # total_run_time += runtime
    # total_ml_run_time += ml_runtime
    if goal_found_agent and goal_found_ml:
      confusion_matrix[0][0] += 1
    elif not goal_found_agent and not goal_found_ml:
      confusion_matrix[1][1] += 1
    elif not goal_found_agent and goal_found_ml:
      confusion_matrix[1][0] += 1
    elif goal_found_agent and not goal_found_ml:
      confusion_matrix[0][1] += 1
    else:
      print("NOOB")
    print(i,'sample------------------')
  accuracy = (confusion_matrix[0][0] + confusion_matrix[1][1]) / confusion_matrix.sum()
  recall = confusion_matrix[0][0]/(confusion_matrix[0][0] + confusion_matrix[0][1])
  precision = confusion_matrix[0][0]/(confusion_matrix[0][0] + confusion_matrix[1][0])
  
  print('avg normal ', total_path_length/solved_agent)
  print('avg ml ', total_ml_path_length/solved_ml_agent)
  return confusion_matrix, accuracy, precision, recall, total_path_length/solved_agent, total_ml_path_length/solved_ml_agent
#traj len, planning time

In [301]:
def write_to_file(states, file_name):
       try:
          print('number of states apend',len(states))
          joblib.dump(states, file_name, compress=3)
       except OverflowError:  
            print ("OverFlow Exception Raised.")

In [ ]:
number_of_tests = 30
sum_confusion_matrix = np.array([[0, 0], 
                            [0, 0]])
sum_accuracy, sum_precision, sum_recall, sum_path_length, sum_ml_path_length = 0, 0, 0, 0, 0
graph_data = []
for i in range(number_of_tests):
  print('test no', i)
  confusion_matrix, accuracy, precision, recall, path_length, ml_path_length = test_ml_agent(number_of_mazes=100)
  graph_data.append([i, confusion_matrix, accuracy, precision, recall, path_length, ml_path_length])
  # sum_confusion_matrix += confusion_matrix
  # sum_accuracy += accuracy
  # sum_precision += precision
  # sum_recall += recall
  # sum_path_length += path_length
  # sum_ml_path_length += ml_path_length



# avg_confusion_matrix = sum_confusion_matrix / number_of_tests
# avg_accuracy = sum_accuracy / number_of_tests
# avg_precision = sum_precision / number_of_tests
# avg_recall = sum_recall / number_of_tests
# avg_path_length = sum_path_length / number_of_tests
# avg_ml_path_length = sum_ml_path_length / number_of_tests
# print('avg')
# print(avg_confusion_matrix, avg_accuracy, avg_precision, avg_recall)

In [ ]:
file_name = 'proj1_cnn_data'
write_to_file(graph_data, file_name)


In [ ]:
data = joblib.load('proj1_cnn_data')
print(len(data))